### Make OpenEO connection

In [3]:
import openeo


connection = openeo.connect("https://openeofed.dataspace.copernicus.eu/").authenticate_oidc()

Authenticated using refresh token.


In [5]:

#Get desired data


EXTENT = dict(zip(["west", "south", "east", "north"], [5.19, 51.25, 5.20, 51.26]))
EXTENT['crs'] = "EPSG:4326"

STARTDATE = '2020-11-01'
ENDDATE = '2020-12-31'

# Set OpenEO classification UDF context based on settings
CONTEXT = {
    "startdate": STARTDATE,  # Required
    "enddate": ENDDATE,      # Required
}

input_cube1 = connection.load_collection(
        collection_id = "SENTINEL2_L2A",
        spatial_extent=EXTENT,
        bands = ["B02", "B03"],
        temporal_extent=[STARTDATE, ENDDATE],
    )

input_cube2 = connection.load_collection(
        collection_id = "AGERA5",
        spatial_extent=EXTENT,
        bands=["temperature-mean", "precipitation-flux"],
        temporal_extent=[STARTDATE, ENDDATE],
    )

cube = input_cube1.merge_cubes(input_cube2)

cube.execute_batch(outputfile = 'test.nc',
                        description='world cereal data collection',
                        job_options={"split_strategy": "crossbackend"})


Preflight process graph validation raised: [InternalValidationFailure] Validation failed: BackendLookupFailureException(status_code=400, code='BackendLookupFailure', message="Collections across multiple backends ({'terrascope', 'cdse'}): {'SENTINEL2_L2A', 'AGERA5'}.", id='r-2405172f25fa4f8bb7f69ca237bd5681')


0:00:00 Job 'agg-pj-20240517-093353': send 'start'
0:00:38 Job 'agg-pj-20240517-093353': running (progress 0%)
0:00:44 Job 'agg-pj-20240517-093353': running (progress 0%)
0:00:51 Job 'agg-pj-20240517-093353': running (progress 0%)
0:01:00 Job 'agg-pj-20240517-093353': running (progress 0%)
0:01:11 Job 'agg-pj-20240517-093353': running (progress 0%)
0:01:25 Job 'agg-pj-20240517-093353': running (progress 0%)
0:01:43 Job 'agg-pj-20240517-093353': running (progress 0%)
0:02:03 Job 'agg-pj-20240517-093353': running (progress 0%)
0:02:34 Job 'agg-pj-20240517-093353': running (progress 50%)
0:03:07 Job 'agg-pj-20240517-093353': running (progress 50%)
0:03:46 Job 'agg-pj-20240517-093353': running (progress 50%)
0:04:34 Job 'agg-pj-20240517-093353': running (progress 50%)
0:05:34 Job 'agg-pj-20240517-093353': running (progress 50%)
0:06:36 Job 'agg-pj-20240517-093353': running (progress 50%)
0:07:38 Job 'agg-pj-20240517-093353': running (progress 50%)
0:08:44 Job 'agg-pj-20240517-093353': runn

<BatchJob job_id='agg-pj-20240517-093353'>

### Check reference

In [6]:
cube